In [250]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta

pd.options.display.max_columns = 5000
pd.options.display.max_rows = 5000

In [172]:
# Load Dataset

# Renzo's Path
dataset = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/complete_final_df.csv')

/Users/renzomaldonado/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (42,43,44,45,46,47,49,51,52,53,54,55,58,59,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,86,87,88,89,90,91,92,93,94,96,97,98,103) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [173]:
# Creating variable 'was_match' to know which rows are matches (real games) and which rows are not

dataset.loc[dataset['Min'].isnull(), 'was_match'] = 0
dataset.loc[dataset['Min'].isnull() == False, 'was_match'] = 1

In [216]:
# Filling in some missing values

dataset.loc[dataset['name'].isnull(), 'Prog'] = dataset['PlayerName']

# [Min, Gls, Ast, PK, PKatt, Sh, SoT, CrdY, CrdR, Touches, Press, Tkl, Int, Blocks, xG, npxG, xA, SCA, GCA, Prog, Carries, Prog.1, Succ, Att.1, \
# Fls, Off, Crs, TklW, OG, PKwon, PKcon, Won, Loss, Draw] all have 150755 NaN's, of which none are when matches are played, so we equal those \
# NaN's in these variables to 0
dataset.loc[dataset['Min'].isnull(), 'Min'] = 0
dataset.loc[dataset['Gls'].isnull(), 'Gls'] = 0
dataset.loc[dataset['Ast'].isnull(), 'Ast'] = 0
dataset.loc[dataset['PK'].isnull(), 'PK'] = 0
dataset.loc[dataset['PKatt'].isnull(), 'PKatt'] = 0
dataset.loc[dataset['Sh'].isnull(), 'Sh'] = 0
dataset.loc[dataset['SoT'].isnull(), 'SoT'] = 0
dataset.loc[dataset['CrdY'].isnull(), 'CrdY'] = 0
dataset.loc[dataset['CrdR'].isnull(), 'CrdR'] = 0
dataset.loc[dataset['Touches'].isnull(), 'Touches'] = 0
dataset.loc[dataset['Press'].isnull(), 'Press'] = 0
dataset.loc[dataset['Tkl'].isnull(), 'Tkl'] = 0
dataset.loc[dataset['Int'].isnull(), 'Int'] = 0
dataset.loc[dataset['Blocks'].isnull(), 'Blocks'] = 0
dataset.loc[dataset['xG'].isnull(), 'xG'] = 0
dataset.loc[dataset['npxG'].isnull(), 'npxG'] = 0
dataset.loc[dataset['xA'].isnull(), 'xA'] = 0
dataset.loc[dataset['SCA'].isnull(), 'SCA'] = 0
dataset.loc[dataset['GCA'].isnull(), 'GCA'] = 0
dataset.loc[dataset['Prog'].isnull(), 'Prog'] = 0
dataset.loc[dataset['Carries'].isnull(), 'Carries'] = 0
dataset.loc[dataset['Prog.1'].isnull(), 'Prog.1'] = 0
dataset.loc[dataset['Succ'].isnull(), 'Succ'] = 0
dataset.loc[dataset['Att.1'].isnull(), 'Att.1'] = 0
dataset.loc[dataset['Fls'].isnull(), 'Fls'] = 0
dataset.loc[dataset['Off'].isnull(), 'Off'] = 0
dataset.loc[dataset['Crs'].isnull(), 'Crs'] = 0
dataset.loc[dataset['TklW'].isnull(), 'TklW'] = 0
dataset.loc[dataset['OG'].isnull(), 'OG'] = 0
dataset.loc[dataset['PKwon'].isnull(), 'PKwon'] = 0
dataset.loc[dataset['PKcon'].isnull(), 'PKcon'] = 0
dataset.loc[dataset['Won'].isnull(), 'Won'] = 0
dataset.loc[dataset['Loss'].isnull(), 'Loss'] = 0
dataset.loc[dataset['Draw'].isnull(), 'Draw'] = 0

In [243]:
# Assigning Dummy Variables for player position from 'Position:'

dataset.loc[dataset['Position:'].isnull(), 'Position:'] = ''
dataset['defender'] = np.where(dataset['Position:'].str.contains('Defender'), 1, 0)
dataset['attacker'] = np.where(dataset['Position:'].str.contains('attack'), 1, 0)
dataset['midfielder'] = np.where(dataset['Position:'].str.contains('midfield'), 1, 0)
dataset['goalkeeper'] = np.where(dataset['Position:'].str.contains('Goalkeeper'), 1, 0)


In [254]:
# Creating the Column Age from current_date and birth

dataset['age'] = round((pd.to_datetime(dataset['current_date']) - pd.to_datetime(dataset['Birth'])) / timedelta(days=365), 0)

In [273]:
# Creating 'retired' dummy variable

dataset['retired'] = np.where(dataset['Retired since:'].isnull(), 0, 1)

In [276]:
dataset.head()


,name,FBRefID,week,year,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,index,Season,Injury,from,until,Days,Games missed,TMId,TmPos,TMID,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,Last club:,Most games for:,Unknown since:,Date of birth:,Foot:,Player agent:,Current club:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_date,current_year,current_week,agg_week,agg_year,was_match,defender,attacker,midfielder,goalkeeper,age,retired
0,Antonio-Cabrini,7e411a43,22.0,1978.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,1978.0,1.0,0,0,0,0,NaN,0
1,Hugo-Sanchez,8378e03e,22.0,1978.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,1978.0,1.0,0,0,0,0,NaN,0
2,Juanito,2484e4d7,22.0,1978.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,1978.0,1.0,0,0,0,0,NaN,0
3,Luis-Arconada,9714ce1c,22.0,1978.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,1978.0,1.0,0,0,0,0,NaN,0
4,Manfred-Kaltz,5f299098,22.0,1978.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,1978.0,1.0,0,0,0,0,NaN,0


In [292]:
# Getting features for modelling

dataset_for_model = dataset[['name', 'FBRefID', 'TMId', 'current_date', 'week', 'year', 'was_match', 'Min', 'Gls', 'Ast', 'PK', 'PKatt','Sh', \
    'SoT', 'CrdY', 'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA', 'SCA', 'Cmp', 'Att', 'Prog', 'Carries', 'Prog.1', \
    'Att.1', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG', 'PKwon', 'PKcon', 'Won', 'Loss', 'Draw', 'Injury', 'injury_year', 'injury_week', \
    'release_week', 'Days', 'Games missed', 'attacker', 'midfielder', 'defender', 'goalkeeper', 'age', 'Height', 'Weight', 'Foot', 'FootAbility', \
    'Caps/Goals:', 'retired']]

In [296]:
# Sorting values in dataset_for_model

# dataset_for_model = dataset_for_model[dataset_for_model['year'] > 2016]
dataset_for_model_final = dataset_for_model.sort_values(['FBRefID', 'year', 'week'])

In [297]:
# dataset_for_model_final[dataset_for_model_final['FBRefID'] == '71672fa0']

In [298]:
# Saving dataset_for_model_final to Google Drive

# Renzo's path
dataset_for_model_final.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/dataset_for_model_final.csv', index=False)